# Compute Spectral indices



In [1]:
import glob, os
import numpy as np
import pandas as pd
import rasterio
from rasterio import plot
from rasterio.plot import show
from pathlib import Path

## Choose which spectral index you want to compute

### NDVI - Normalized Difference Vegetation Index

NDVI is used to outline the presence of vegetation. It is used to indicate relative density, or the amount, of the green vegetation present in the image. This index uses reflectance from a red band around 0.66 μm and a near-Infrared band around 0.86 μm. The red band is found in the absorption region of the chlorophyll, while the near-IR band is used in high reflectance plateau of the vegetation canopies. These two bands sense different depths over the vegetation canopies.

$$ NDVI = \frac{NIR - RED}{NIR + RED}$$

### NDWI - Normalized Difference Water Index

NDWI is used to differentiate water from the dry land or rather most suitable for water body mapping. Water bodies have a low radiation and strong absorbability in the visible infrared wavelengths range. NDWI uses near Infra-red and green bands of remote sensing images based on the occurrence. It can boost the water information efficiently in most of the cases. It’s subtle in land built-up and often ends up in overestimated water bodies.

$$ NDWI = \frac{NIR - MIR}{NIR + MIR}$$

### NDSI - Normalized Difference Snow Index

NDSI is used to delineate the presence of snow/ice. It is a standardized ratio of the difference in the reflectance in the bands that take advantage of unique signature and the spectral difference to indicate snow from the surrounding features and even clouds.

$$ NDSI = \frac{GREEN - SWIR}{GREEN + SWIR}$$

### BRIGHTNESS

Brightness provide complementary information improving the discrimination between crop and no-crop areas.

$$ Brightness =\sqrt{GREEN^{2}+RED^{2}+NIR^{2}+SWIR^{2}} $$


Source : https://eos.com/make-an-analysis/index-stack/

In [3]:
index_name   = 'NDVI'

if index_name == 'NDVI':
    bands_needed = [4,8]

elif index_name == 'NDWI':
    bands_needed = [8,12]

elif index_name == 'NDWI':
    bands_needed = [3,11]

print(f'You chose to compute {index_name} !')

You chose to compute NDVI !


## Set paths for input and output directories

In [4]:
# Directory for all work files

grp_letter = 'X'
#work_path = f'X:/GROUP_{grp_letter}/WORK/'
work_path = f'/Volumes/nbdid-sst-lbrat2104/GROUP_{grp_letter}/WORK/'

# ----- #
# INPUT #
# ----- #
 
im_path = f'{work_path}3_L2A_MASKED/'

# ------ #
# OUTPUT #
# ------ #

index_path = f'{work_path}{index_name}/'

Path(index_path).mkdir(parents=True, exist_ok=True)

print(f'General work path is set to   : {work_path}')
print(f'Spectral index path is set to : {index_path}')

General work path is set to   : /Volumes/nbdid-sst-lbrat2104/GROUP_X/WORK/
Spectral index path is set to : /Volumes/nbdid-sst-lbrat2104/GROUP_X/WORK/NDVI/


## Check if you have the necessary spectral bands

In [5]:

for band_nb in bands_needed:
    print(f'We need band {band_nb} to compute {index_name}')
    list_im = glob.glob(f'{im_path}*_B*{band_nb}*.tif')
    print(f'--> {len(list_im)} images are found \n')

print('Check if the number of images found for each band is the same !!')


We need band 4 to compute NDVI
--> 12 images are found 

We need band 8 to compute NDVI
--> 12 images are found 

Check if the number of images found for each band is the same !!


## Compute Spectral Indices using 2 spectral bands

- NDVI
- NDWI
- NDSI

In [6]:

first_band  = bands_needed[0]
second_band = bands_needed[1]

nodata_val = -10000

list_im = glob.glob(f'{im_path}*_B*{first_band}*.tif')

for im_file_1 in list_im:

    # Get date of image
    date = os.path.basename(im_file_1)[7:7+15]
    
    # Find second band needed
    im_file_2 = glob.glob(f'{im_path}*{date}*_B*{second_band}*.tif')[0]
    im_file_2 = im_file_2.replace('\\','/')
    
    index_file = f'{index_path}{os.path.basename(im_file_1)[0:22]}_{index_name}.tif'

    if not os.path.isfile(index_file):

        # Open first band and update metadata

        src = rasterio.open(im_file_1, 'r')

        profile = src.profile
        profile.update(dtype=rasterio.float64)

        band_1 = src.read(1)
        src.close()

        # Open second band

        src = rasterio.open(im_file_2, 'r')
        band_2 = src.read(1)
        src.close()

        # Convert no-data value into Numpy NaN

        band_1 = band_1.astype(np.float)
        band_1[band_1 == nodata_val] = np.nan

        band_2 = band_2.astype(np.float)
        band_2[band_2 == nodata_val] = np.nan

        # Compute Spectral Index

        if index_name == 'NDVI':
            # band_1 = RED / B04
            # band_2 = NIR / B08
            index_arr = (band_2 - band_1) / (band_2 + band_1)

        elif index_name == 'NDSI':
            # band_1 = RED / B04
            # band_2 = NIR / B08
            index_arr = (band_2 - band_1) / (band_2 + band_1)


        # Write Spectral Index image into GeoTIFF

        dst = rasterio.open(index_file, "w", **profile)
        dst.write(index_arr,1)
        dst.close()

        print(f'A new {index_name} file is created : {index_file}')
    
    else:
        print(f'--> {index_file} - already exists')


--> /Volumes/nbdid-sst-lbrat2104/GROUP_X/WORK/NDVI/T31UFS_20200116T105309_NDVI.tif - already exists
--> /Volumes/nbdid-sst-lbrat2104/GROUP_X/WORK/NDVI/T31UFS_20200212T104049_NDVI.tif - already exists
--> /Volumes/nbdid-sst-lbrat2104/GROUP_X/WORK/NDVI/T31UFS_20200316T104709_NDVI.tif - already exists
--> /Volumes/nbdid-sst-lbrat2104/GROUP_X/WORK/NDVI/T31UFS_20200417T104021_NDVI.tif - already exists
--> /Volumes/nbdid-sst-lbrat2104/GROUP_X/WORK/NDVI/T31UFS_20200520T105031_NDVI.tif - already exists
--> /Volumes/nbdid-sst-lbrat2104/GROUP_X/WORK/NDVI/T31UFS_20200621T103629_NDVI.tif - already exists
--> /Volumes/nbdid-sst-lbrat2104/GROUP_X/WORK/NDVI/T31UFS_20200719T105031_NDVI.tif - already exists
--> /Volumes/nbdid-sst-lbrat2104/GROUP_X/WORK/NDVI/T31UFS_20200813T104629_NDVI.tif - already exists
--> /Volumes/nbdid-sst-lbrat2104/GROUP_X/WORK/NDVI/T31UFS_20200914T104031_NDVI.tif - already exists
--> /Volumes/nbdid-sst-lbrat2104/GROUP_X/WORK/NDVI/T31UFS_20201019T103959_NDVI.tif - already exists


## Compute Spectral Indices using 4 spectral bands

- BRIGHTNESS

In [7]:
### TELL ME IF YOU NEED THIS SCRIPT FOR YOUR POSTER